In [ ]:
%load_ext autoreload
%autoreload 2
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
# Set here the right device id!
%env CUDA_VISIBLE_DEVICES=2


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [ ]:


from vllm import LLM, SamplingParams
from encourage.llm import BatchInferenceRunner
from encourage.prompts.prompt_collection import PromptCollection
from encourage.metrics.non_answer_critic import NonAnswerCritic
from encourage.prompts.context import Context
from encourage.prompts.meta_data import MetaData
model_name= "meta-llama/Meta-Llama-3.1-8B-Instruct"


In [ ]:
llm = LLM(model=model_name, gpu_memory_utilization=0.8)
sampling_params = SamplingParams(temperature=0.5, max_tokens=1000)

WARNING 12-03 16:36:40 arg_utils.py:930] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-03 16:36:40 config.py:1010] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 12-03 16:36:40 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 12-03 16:36:54 model_runner.py:1025] Loading model weights took 14.9888 GB
INFO 12-03 16:36:55 gpu_executor.py:122] # GPU blocks: 11039, # CPU blocks: 2048
INFO 12-03 16:36:59 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-03 16:36:59 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 12-03 16:37:29 model_runner.py:1456] Graph capturing finished in 31 secs.


In [ ]:
sys_prompts = "You are an helpful AI that only speaks like a pirat"
user_prompts = ["User prompt 1", "User prompt 2"] * 5
contexts = [Context.from_prompt_vars({"key1": "value1"}, {"key2": "value2"}) for _ in range(5)]
meta_datas = [MetaData({"meta": "data1"}), MetaData({"meta": "data2"})] * 5

prompt_collection = PromptCollection.create_prompts(
    sys_prompts=sys_prompts,
    user_prompts=user_prompts,
    contexts=contexts,
    meta_datas=meta_datas,
    model_name=model_name,
    template_name="llama3_custom.j2"
)


In [44]:
from encourage.llm.response import Response
from encourage.llm import ResponseWrapper


responses = ResponseWrapper([
            Response(
                request_id="1",
                prompt_id="p1",
                sys_prompt="System prompt example.",
                user_prompt="User prompt example.",
                response="This is a generated answer.",
                conversation_id=1,
                meta_data={
                    "reference_answer": "This is the reference answer.",
                    "reference_document": ["doc1"],  # Required field for MRR
                },
                context={
                    "contexts": [  # Required field for MRR
                        {"content": "Here is an example content", "document": "doc1", "score": 1.0},
                        {"content": "Here is example content", "document": "doc2", "score": 0.5},
                    ]
                },
                arrival_time=0.0,
                finished_time=1.0,
            ),
            Response(
                request_id="2",
                prompt_id="p2",
                sys_prompt="Another system prompt.",
                user_prompt="Another user prompt.",
                response="Another generated answer.",
                conversation_id=2,
                meta_data={
                    "reference_answer": "Another reference answer.",
                    "reference_document": ["doc2"],  # Required field for MRR
                },
                context={
                    "contexts": [  # Required field for MRR
                        {"content": "Here is an example content", "document": "doc2", "score": 1.0},
                        {"content": "Here is an example content", "document": "doc1", "score": 0.0},
                    ]
                },
                arrival_time=0.0,
                finished_time=1.0,
            ),
        ])

In [58]:
responses = ResponseWrapper([
    Response(
        request_id="0",
        prompt_id="p0",
        sys_prompt="System prompt example.",
        user_prompt="What is the highest peak in Jotunheimen National Park?",
        response="The highest peak in Jotunheimen National Park is Glittertind. It is 2,465 metres (8,087 ft) tall.",
        conversation_id=0,
        meta_data={
            "reference_answer": "Galdhøpiggen at 2,469 metres (8,100 ft).",
            "reference_document": ["website1"]
        },
        context={
            "contexts": [
                {"content": "More than 250 peaks rise above an elevation of 1,900 metres (6,200 ft), including Northern Europe's two highest peaks: Galdhøpiggen at 2,469 metres (8,100 ft), and Glittertind at 2,465 metres (8,087 ft).", "document": "website1", "score": 7},
                {"content": "Jotunheimen National Park is part of the larger area Jotunheimen.", "document": "website1", "score": 6}
            ]
        },
        arrival_time=0.0,
        finished_time=1.0
    ),
    Response(
        request_id="1",
        prompt_id="p1",
        sys_prompt="System prompt example.",
        user_prompt="Which counties is Jotunheimen National Park located in?",
        response="Jotunheimen National Park is located in Oslo and Akershus counties.",
        conversation_id=1,
        meta_data={
            "reference_answer": "Innlandet and Vestland counties.",
            "reference_document": ["website1"]
        },
        context={
            "contexts": [
                {"content": "Jotunheimen includes Hurrungane, but Utladalen and its surroundings are within Utladalen Landscape Protection Area.", "document": "website2", "score": 7},
                {"content": "Geographically, it lies in both Innlandet and Vestland counties.", "document": "website1", "score": 5}
            ]
        },
        arrival_time=0.0,
        finished_time=1.0
    ),
    Response(
        request_id="2",
        prompt_id="p2",
        sys_prompt="System prompt example.",
        user_prompt="What wildlife can be found in Jotunheimen National Park?",
        response="Wildlife in Jotunheimen National Park includes lynx, wolves, brown bears, and musk oxen.",
        conversation_id=2,
        meta_data={
            "reference_answer": "Lynx, moose, Norwegian red deer, reindeer, roe deer, and wolverine.",
            "reference_document": ["website1"]
        },
        context={
            "contexts": [
                {"content": "Wildlife in the park include the lynx, moose, Norwegian red deer, reindeer, roe deer, and wolverine.", "document": "website1", "score": 7},
                {"content": "Most lakes and rivers hold trout.", "document": "website2", "score": 3}
            ]
        },
        arrival_time=0.0,
        finished_time=1.0
    ),
    Response(
        request_id="3",
        prompt_id="p3",
        sys_prompt="System prompt example.",
        user_prompt="What archaeological discovery was made in Jotunheimen in 2020?",
        response="In 2020 a significant discovery was made in Jotunheimen by researchers. They found a 1,500-year-old Viking arrowhead, which was made of iron and locked away in a glacier.",
        conversation_id=3,
        meta_data={
            "reference_answer": "A 1,500-year-old Viking arrowhead was discovered in Jotunheimen.",
            "reference_document": ["website1"]
        },
        context={
            "contexts": [
                {"content": "The arrowhead made of iron was revealed with its cracked wooden shaft and a feather, is 17 cm long and weighs just 28 grams.", "document": "website2", "score": 6},
                {"content": "In February 2020, Secrets of the Ice Program researchers discovered a 1,500-year-old Viking arrowhead dating back to the Germanic Iron Age and locked in a glacier in southern Norway caused by the climate change in the Jotunheimen Mountains.", "document": "website1", "score": 4}
            ]
        },
        arrival_time=0.0,
        finished_time=1.0
    ),
    Response(
        request_id="4",
        prompt_id="p4",
        sys_prompt="System prompt example.",
        user_prompt="What did the Royal Road decree of the 15th century require?",
        response="The Royal Road decree of the 15th century required the residents of Lom to keep the mountain crossing passable to the middle of the Sognefjell.",
        conversation_id=4,
        meta_data={
            "reference_answer": "The residents of Lom must keep the mountain crossing passable to the middle of the Sognefjell. This is so that traders coming from the north can reach their trading town Bergen.",
            "reference_document": ["website1"]
        },
        context={
            "contexts": [
                {"content": "A 'Royal Road' decree from the 15th century required that the residents of Lom must keep the mountain crossing passable to the middle of the Sognefjell, allowing traders to pass.", "document": "website1", "score": 8},
                {"content": "Caravans carried farm products down the mountains and returned with salt, iron, cloth and lutefisk.", "document": "website1", "score": 5}
            ]
        },
        arrival_time=0.0,
        finished_time=1.0
    ),
    Response(
        request_id="5",
        prompt_id="p5",
        sys_prompt="System prompt example.",
        user_prompt="How many tourists hike Besseggen each year?",
        response="I'm sorry but I do not have any information about how many tourists come to Jotunheimen to hike the Besseggen ridge.",
        conversation_id=5,
        meta_data={
            "reference_answer": "An estimated 60,000 people walk the Besseggen ridge every year.",
            "reference_document": ["website1"]
        },
        context={
            "contexts": [
                {"content": "More than 250 peaks rise above an elevation of 1,900 metres (6,200 ft), including Northern Europe's two highest peaks: Galdhøpiggen at 2,469 metres (8,100 ft), and Glittertind at 2,465 metres (8,087 ft).", "document": "website1", "score": 7},
                {"content": "Jotunheimen National Park is part of the larger area Jotunheimen.", "document": "website1", "score": 6}
            ]
        },
        arrival_time=0.0,
        finished_time=1.0
    ),
])

In [59]:
runner = BatchInferenceRunner(llm, sampling_params)
# responses = runner.run(prompt_collection)
responses.print_response_summary()

--------------------------------------------------
🧑‍💻 User Prompt:
What is the highest peak in Jotunheimen National Park?
📚 Added Context keys: contexts (See Template for details.)

💬 Response:
The highest peak in Jotunheimen National Park is Glittertind. It is 2,465 metres (8,087 ft) tall.

🤖 System Prompt:
System prompt example.

🗂️ Metadata: {'reference_answer': 'Galdhøpiggen at 2,469 metres (8,100 ft).', 'reference_document': ['website1']}
🆔 Request ID: 0
🆔 Prompt ID: p0
🆔 Conversation ID: 0
⏳ Processing Time: 1.0 seconds

--------------------------------------------------
🧑‍💻 User Prompt:
Which counties is Jotunheimen National Park located in?
📚 Added Context keys: contexts (See Template for details.)

💬 Response:
Jotunheimen National Park is located in Oslo and Akershus counties.

🤖 System Prompt:
System prompt example.

🗂️ Metadata: {'reference_answer': 'Innlandet and Vestland counties.', 'reference_document': ['website1']}
🆔 Request ID: 1
🆔 Prompt ID: p1
🆔 Conversation ID: 1
⏳

In [78]:

from encourage.metrics.context_recall import ContextRecall
from encourage.metrics.context_precision import ContextPrecision
from encourage.metrics.answer_similarity import AnswerSimilarity
from encourage.metrics.answer_relevance import AnswerRelevance
from encourage.metrics.answer_faithfulness import AnswerFaithfulness

# metric = ContextRecall(runner)
# metric = AnswerSimilarity("all-mpnet-base-v2")
metric = AnswerFaithfulness(runner)

test = metric(responses)

Processed prompts: 100%|██████████| 6/6 [00:03<00:00,  1.63it/s, est. speed input: 1234.91 toks/s, output: 46.82 toks/s]


In [76]:
a = ('simpler_statements', ['I do not have any information about how many tourists come to Jotunheimen to hike the Besseggen ridge.'])
b,c = a
b

'simpler_statements'

In [79]:
test

{'score': 1.0,
 'raw': [1.0, nan, nan, nan, nan, nan],
 'supported': [2, 0, 0, 0, 0, 0],
 'total': [2, 0, 0, 0, 0, 0],
 'claims': [[Verdict(statement='I do not have any information about tourists coming to Jotunheimen to hike the Besseggen ridge.', reason='The context does not mention anything about tourists coming to Jotunheniem to hike the Besseggen ridge.', verdict=1),
   Verdict(statement='I do not have any information about the number of tourists coming to Jotunheimen to hike the Besseggen ridge.', reason='The context does not mention anything about tourists coming to Jotunheniem to hike the Besseggen ridge.', verdict=1)],
  [],
  [],
  [],
  [],
  []]}

AttributeError: 'MetricOutput' object has no attribute 'print_metric_summary'